In [1]:
import numpy as np
import keras.utils as image
import matplotlib.pyplot as plt
from PIL import Image
import skimage.measure
from scipy.ndimage import uniform_filter
import math
import sys
import os
import time

import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
filename = 'zebra'
img = image.load_img('./'+filename+'.jpg')
height,width = img.size
channels = 3
img = image.img_to_array(img)
tex_pixels = tf.squeeze(img)/255.

In [5]:
# for regular algorithm
max_sigma = 235
# max_sigma = height*2 - 1
sigma_sequence = [1.0, 1.5, 2.0, 4.0, 7.0, 10.0]
increment = 5.0
increment_steps = [5.0, 10.0, 20.0, 40.0, 50.0]
increment_index = 0
step_counter = 0
current_sigma = sigma_sequence[-1] + increment
while current_sigma <= max_sigma:
    sigma_sequence.append(current_sigma)
    step_counter += 1
    if step_counter % 5 == 0 and increment_index < len(increment_steps) - 1:
        increment_index += 1
        increment = increment_steps[increment_index]
    current_sigma += increment
    if current_sigma >= max_sigma:
        sigma_sequence.append(max_sigma)
        break

In [6]:
# for tight sigma list
max_sigma = img.shape[0] // 2
sigma_sequence = [1.0]
increment_steps = [1.0, 5.0]
current_increment = increment_steps[0]

while sigma_sequence[-1] + current_increment <= max_sigma:
    current_sigma = min(sigma_sequence[-1] + current_increment, max_sigma)

    if 1 <= current_sigma < max_sigma//3:
        current_increment = increment_steps[0]
    else:
        current_increment = increment_steps[1]

    sigma_sequence.append(current_sigma)

print(sigma_sequence)

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 85.0, 90.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140.0, 145.0, 150.0, 155.0, 160.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190.0, 195.0, 200.0, 205.0, 210.0, 215.0, 220.0, 225.0, 230.0, 235.0, 240.0]


In [7]:
start = time.time()
tsg_lists = {}
for sigma in sigma_sequence:
    kernel_size = int(4 * sigma - 1)
    current_height = int(kernel_size / 2)
    current_width = int(kernel_size / 2)
    tsg_current = np.array(
                     [[np.exp(-((i_ref_p)**2 + (j_ref_p)**2) / (2*sigma**2)) \
                    for j_ref_p in range(-current_width, current_width + 1)] \
                    for i_ref_p in range(-current_height, current_height + 1)])

    tsg_current = tsg_current / np.sum(tsg_current)
    tsg_current = np.expand_dims(tsg_current, axis=-1)
                                      # Add a third dimension for 'in_channels'
    tsg_current = np.expand_dims(tsg_current, axis=-1)
                                    # Add a fourth dimension for 'out_channels'
    tsg_current = np.repeat(tsg_current, 3, axis=-1)
    tsg_current_tf = tf.convert_to_tensor(tsg_current, dtype=tf.float32)
    tsg_lists[sigma] = tsg_current_tf
print(time.strftime("%H:%M:%S", time.gmtime(time.time()-start)))

00:00:44


In [8]:
mean_tex_all = []
var_tex_all = []
start = time.time()
for sigma in sigma_sequence:
    tsg = tsg_lists[sigma]
    tex = tf.expand_dims(tex_pixels, axis=0)  # Add batch dimension
    kernel_size = len(tsg)
    padding_size = int((kernel_size + 1) / 2) - 1
    padding = tf.constant([[0, 0], [padding_size, padding_size], [padding_size, padding_size], [0, 0]])
    tex_padded = tf.pad(tex, padding, "SYMMETRIC")
    # Convolution
    # print(time.strftime("%H:%M:%S", time.gmtime(time.time()-start)))
    mean_tex = tf.nn.conv2d(tex_padded, tsg, strides=[1, 1, 1, 1], padding='VALID')
    # print(time.strftime("%H:%M:%S", time.gmtime(time.time()-start)))
    mean_tex_padded = tf.pad(mean_tex, padding, "SYMMETRIC")
    # print(time.strftime("%H:%M:%S", time.gmtime(time.time()-start)))
    squared_diff_tex = (tex_padded - mean_tex_padded) ** 2
    # print(time.strftime("%H:%M:%S", time.gmtime(time.time()-start)))
    var_tex = tf.nn.conv2d(squared_diff_tex, tsg, strides=[1, 1, 1, 1], padding='VALID')
    print(time.strftime("%H:%M:%S", time.gmtime(time.time()-start)))
    var_tex = tf.maximum(var_tex, 0)
    mean_tex_all.append(mean_tex)
    var_tex_all.append(var_tex)

mean_tex_all = np.concatenate(mean_tex_all,axis=0)
var_tex_all = np.concatenate(var_tex_all,axis=0)
wd_sigma_list = (mean_tex_all[1:,:,:,:] - mean_tex_all[:-1,:,:,:])**2 + var_tex_all[1:,:,:,:] + var_tex_all[:-1,:,:,:] -\
                                            2*tf.math.sqrt(var_tex_all[1:,:,:,:]*var_tex_all[:-1,:,:,:])
wd_sigma_list = tf.math.reduce_mean(wd_sigma_list,axis=3)
print(time.strftime("%H:%M:%S", time.gmtime(time.time()-start)))

00:00:00
00:00:00
00:00:00
00:00:01
00:00:01
00:00:02
00:00:04
00:00:05


KeyboardInterrupt: 

In [ ]:
wd_sigma_list_np = wd_sigma_list.numpy()
np.save(filename+'_wd_sigma_list.npy', wd_sigma_list_np)
print(filename+"_wd_sigma_list has been saved")

In [ ]:
#single point
wd_sigma_420_20 = wd_sigma_list[:,300,300]
plt.figure(figsize=(16,9))
plt.plot(wd_sigma_420_20,marker='*')

In [ ]:
# multiple points
wd_sigma_300_300 = wd_sigma_list[:,300,300]
wd_sigma_240_240 = wd_sigma_list[:,240,240]
wd_sigma_40_440 = wd_sigma_list[:,40,440]
wd_sigma_120_360 = wd_sigma_list[:,120,360]

fig, axs = plt.subplots(2, 2, figsize=(12, 6))

axs[0, 0].plot(sigma_values, wd_sigma_300_300, marker='+',\
               color='blue', label='[300, 300]')
axs[0, 0].legend()
axs[0, 0].grid(True)

axs[0, 1].plot(sigma_values, wd_sigma_240_240, marker='+',\
               color='green', label='[240,240]')
axs[0, 1].legend()
axs[0, 1].grid(True)

axs[1, 0].plot(sigma_values, wd_sigma_40_440, marker='+',\
               color='red', label='[40, 440]')
axs[1, 0].legend()
axs[1, 0].grid(True)


axs[1, 1].plot(sigma_values, wd_sigma_120_360, marker='+',\
               color='purple', label='[120,360]')
axs[1, 1].legend()
axs[1, 1].grid(True)

fig.supxlabel(r'$\sigma_i$', fontsize=18)
fig.supylabel(r'$D_{\cdot,\cdot, \sigma_i, \sigma_{i+1}}$', fontsize=18)
plt.tight_layout()
plt.savefig('zebra_tight_sequence.png',bbox_inches='tight')
plt.show()

In [ ]:
# 3rd double prime version:
k = len(sigma_sequence)//5
threshold_low = 0.15
threshold_high = 0.35
threshold_low_map = np.full((height, width), threshold_low)
threshold_high_map = np.full((height, width), threshold_high)
overall_max_values = np.max(wd_sigma_list, axis=0)
max_indices = np.argmax(wd_sigma_list, axis=0)
is_max_in_top5 = max_indices <= k
# Compute threshold maps
threshold_map = np.where(is_max_in_top5, threshold_low_map, threshold_high_map)

# Compute ratios
ratios = wd_sigma_list / overall_max_values
ratios = ratios[k:, :, :]
sigma_sequence_final = [0.] + sigma_sequence[k:]

# Compute sigma map for thresh
indices = np.argmax(ratios >= threshold_map, axis=0)
sigma_map = np.take(sigma_sequence_final, indices)
# Apply normalization
# saliency_map = create_saliency_map(sigma_map, 50)
# saliency_map = (saliency_map * 255).astype(np.uint8)

In [ ]:
salmap = np.copy(saliency_map)
salmap[salmap < 0.93] = 0

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(8,8),width_ratios=[6,0.1])
plt.tight_layout()
im=ax[0].imshow(sigma_map, cmap='gray')
                  # Use 'bone_r' to invert grayscale color mapping cmap='bone'
ax[0].axis('off')
plt.colorbar(im,cax=fig.add_subplot(ax[1]),shrink=0.7)